In [215]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz
!tar xf spark-3.0.0-bin-hadoop3.2.tgz

In [216]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"
# install findspark using pip
!pip install -q findspark

In [217]:
import findspark
findspark.init()

In [218]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [219]:
import numpy as np
from bisect import bisect
from pyspark.sql import functions as func
from pyspark.sql.types import IntegerType, DoubleType
from pyspark.sql.functions import concat,col,lit,desc
from operator import itemgetter
import itertools 
from bisect import bisect
import pandas as pd

In [220]:
team = team_a = 'milan'
opponent = 'cagliari'
half = 'first half'
half_split = '2'
player_in_possess = 'all'

name_path = team +"_"+str(opponent)+"_"+str(half).replace(' ','_')+"_"+str(half_split)+"_"+player_in_possess

name_path

'milan_cagliari_first_half_2_all'

In [221]:
# mount your drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [222]:
# loading data as parquet
pre_data_players = spark.read.parquet("/content/drive/MyDrive/Colab Notebooks/FootballDataAnalysis/data/Milan-Cagliari/parquet/frames_milan_cagliari.parquet").drop('index')
pre_pre_df = spark.read.csv('/content/drive/MyDrive/Colab Notebooks/FootballDataAnalysis/data/Milan-Cagliari/parquet/' + 'pitch_control_' + name_path + '.csv', header = True)

pre_pre_df.groupBy('target_x').count().show(100)

+--------+-----+
|target_x|count|
+--------+-----+
|     1.0|  293|
|    -7.0|  273|
|   -18.0|  262|
|    50.0|  230|
|    22.0|  428|
|    38.0|  335|
|    20.0|  405|
|    -4.0|  284|
|    15.0|  383|
|   -11.0|  304|
|    45.0|  250|
|   -31.0|   84|
|    47.0|  238|
|   -21.0|  173|
|   -17.0|  273|
|   -10.0|  300|
|    44.0|  257|
|    25.0|  404|
|   -44.0|   16|
|    26.0|  395|
|   -27.0|  104|
|    17.0|  374|
|   -47.0|    7|
|    41.0|  304|
|    51.0|  229|
|     0.0|  296|
|    52.0|  226|
|    -5.0|  279|
|     9.0|  425|
|    12.0|  421|
|   -45.0|   13|
|    18.0|  392|
|    48.0|  231|
|    10.0|  436|
|   -23.0|  138|
|   -24.0|  125|
|    -3.0|  284|
|    29.0|  415|
|   -37.0|   40|
|    14.0|  384|
|   -19.0|  230|
|    37.0|  326|
|   -48.0|    2|
|   -38.0|   31|
|   -34.0|   72|
|    46.0|  243|
| 10000.0|    6|
|    19.0|  400|
|  -999.0|    3|
|    23.0|  425|
|   -43.0|   19|
|   -20.0|  199|
|    -2.0|  296|
|     5.0|  328|
|   -41.0|   25|
|     6.0|  34

In [223]:
#create parameters input boxes (Databricks)

if opponent == 'milan':
  opponent_f = team_a
else:
  opponent_f = opponent

In [224]:
pre_pre_df = pre_pre_df.withColumnRenamed('player_id','player')

In [225]:
pre_pre_df.where(pre_pre_df['player'].isNull()).show()

+--------+--------+-------------+-------------------+------+------+----+---+---+-----------+--------------------+
|target_x|target_y|ball_distance|slope_fuzzy_compare|player|  time|team|  x|  y|probability|distance_from_target|
+--------+--------+-------------+-------------------+------+------+----+---+---+-----------+--------------------+
|     0.0|     0.0|          0.0|                0.0|  null|3291.0|null|0.0|0.0|        0.0|                 0.0|
+--------+--------+-------------+-------------------+------+------+----+---+---+-----------+--------------------+



In [226]:
from pyspark.sql import functions as func

#avoidPositionOutliers = list(pre_data_players.agg(func.expr('percentile(x, array(0.01))')[0],func.expr('percentile(y, array(0.01))')[0],func.expr('percentile(x, array(0.99))')[0],func.expr('percentile(y, array(0.99))')[0]).collect()[0])
#pre_data_players = pre_data_players.where(((pre_data_players['x'] > avoidPositionOutliers[0])&(pre_data_players['x'] < avoidPositionOutliers[2]))&((pre_data_players['y'] > avoidPositionOutliers[1])&(pre_data_players['y'] < avoidPositionOutliers[3])))

pre_data_players = pre_data_players.withColumnRenamed('team_name','team').withColumnRenamed('position','role')

min_half_time, max_half_time = pre_data_players.where(pre_data_players['half'] == half).groupBy('half').agg(func.min('time'),func.max('time')).collect()[0][1:]

if half_split == '1':
  data_players = pre_data_players.where((pre_data_players['time'] > min_half_time) & (pre_data_players['time'] <= min_half_time+float((max_half_time-min_half_time)/2.0)))
  
elif half_split == '2':
  data_players = pre_data_players.where((pre_data_players['time'] > min_half_time+float((max_half_time-min_half_time)/2.0)) & (pre_data_players['time'] < max_half_time))


In [227]:
from operator import itemgetter
from pyspark.sql import functions as func
from pyspark.sql.functions import lit

import itertools 
import bisect
import numpy as np
from pyspark.sql.types import DoubleType
import pandas as pd

team_a_lower = team_a.lower()
granularity = 0.5
max_ball_distance = 70.0
max_meters_distance_player_can_receive_ball = 30.0
opponent_lower = opponent.lower()

team_a_goalies = data_players.where(data_players['team'] == team_a.title()).where(data_players['role'] == 'Goalkeeper').select('player').distinct().collect()[0]['player']
opponent_goalies = data_players.where(data_players['team'] == opponent.title()).where(data_players['role'] == 'Goalkeeper').select('player').distinct().collect()[0]['player']

goalie_x = data_players.where(data_players['team'] == team_a.title()).where(data_players['role'] == 'Goalkeeper').groupBy('role').avg('x').rdd.max()[1]

team_a_goalies,opponent_goalies,goalie_x

('Maignan', 'Radunovic', 30.176983485692936)

In [228]:
min_x = data_players.select('x').rdd.min()[0]
max_x = data_players.select('x').rdd.max()[0]
min_y = data_players.select('y').rdd.min()[0]
max_y = data_players.select('y').rdd.max()[0]

x_lines = list(np.arange(int(min_x)-1,int(max_x)+1,float(max_x-min_x)/6.0))
y_lines = list(np.arange(int(min_y)-1,int(max_y)+1,float(max_y-min_y)/4.0))

mid_x, mid_y = x_lines[3], float(y_lines[2])

if float(np.abs(goalie_x - max_x)) < float(np.abs(goalie_x - min_x)):
  verse = 'right_to_left'
  opponent_verse = 'left_to_right'
else:
  verse = 'left_to_right'
  opponent_verse = 'right_to_left'

team_a_goalies,verse

('Maignan', 'right_to_left')

In [229]:
pre_pre_df = pre_pre_df.withColumn('time', pre_pre_df['time'].cast(DoubleType()))

In [230]:
possess = pre_pre_df.where(pre_pre_df['target_x'] < -950).select('player','team','time','x','y').distinct().withColumnRenamed('player','player_in_possess').withColumnRenamed('team','team_in_possess')

timings = sorted(possess.select('time').rdd.map(lambda a : a[0]).collect())
timings

[2763.0, 3291.0, 4315.0]

In [231]:
df = pre_pre_df.where(pre_pre_df['target_x'] > -999)
df = df.distinct()

In [232]:
pre_pre_df.groupBy('team').count().show()

+--------+-----+
|    team|count|
+--------+-----+
|    null|    1|
|   Milan|14295|
|Cagliari|11843|
+--------+-----+



In [233]:
verse

'right_to_left'

In [234]:
df_locs = df.select('player','time','team','x','y').distinct()

if verse == 'right_to_left':
  opponent_goal_line = [min_x+5.0, float(max_y+min_y)/2.0]   
else:
  opponent_goal_line = [max_x-5.0, float(max_y+min_y)/2.0]

df_locs = df_locs.withColumn('distance_from_opponent_goal_line', 0.25*func.sqrt((df_locs['x'] - opponent_goal_line[0])**2 + (df_locs['y'] - opponent_goal_line[1])**2) + 0.75*func.sqrt((df_locs['x'] - opponent_goal_line[0])**2))
panda_locs = df_locs.toPandas()

verse,opponent_goal_line

('right_to_left', [-62.30000305175781, -4.549999237060547])

In [235]:
panda_possess = possess.toPandas()

In [236]:
x_ranges = list(np.arange(min_x,max_x,3.0))
y_ranges = list(np.arange(min_y,max_y,3.0))

In [237]:
x_ranges, y_ranges

([-67.30000305175781,
  -64.30000305175781,
  -61.30000305175781,
  -58.30000305175781,
  -55.30000305175781,
  -52.30000305175781,
  -49.30000305175781,
  -46.30000305175781,
  -43.30000305175781,
  -40.30000305175781,
  -37.30000305175781,
  -34.30000305175781,
  -31.300003051757812,
  -28.300003051757812,
  -25.300003051757812,
  -22.300003051757812,
  -19.300003051757812,
  -16.300003051757812,
  -13.300003051757812,
  -10.300003051757812,
  -7.3000030517578125,
  -4.3000030517578125,
  -1.3000030517578125,
  1.6999969482421875,
  4.6999969482421875,
  7.6999969482421875,
  10.699996948242188,
  13.699996948242188,
  16.699996948242188,
  19.699996948242188,
  22.699996948242188,
  25.699996948242188,
  28.699996948242188,
  31.699996948242188,
  34.69999694824219,
  37.69999694824219,
  40.69999694824219,
  43.69999694824219,
  46.69999694824219,
  49.69999694824219,
  52.69999694824219,
  55.69999694824219],
 [-44.5,
  -41.5,
  -38.5,
  -35.5,
  -32.5,
  -29.5,
  -26.5,
  -23.5,


In [238]:
def x_area_handling(x):
  
  if x >= 30:
    return x
  elif x < 30 and x >= 10:
    return int(0.8*float(x))
  elif x < 10 and x >= -20:
    return int(0.5*float(x))
  elif x < -20:
    return 10

x_single_area_prob = 100.0/float(len(x_ranges))
y_single_area_prob = 100.0/float(len(y_ranges))

x_ranges_value_right_to_left = list(map(lambda d : x_area_handling(float(x_single_area_prob)*(len(x_ranges)-x_ranges.index(d))), x_ranges))
x_ranges_value_left_to_right = list(map(lambda d : x_area_handling(float(x_single_area_prob)*(x_ranges.index(d))), x_ranges))
y_ranges_value = list(map(lambda d : 100-1.75*(float(y_single_area_prob)*float(np.abs((mid_y-y_ranges.index(d))))), y_ranges))
                      
if verse == 'left_to_right':
  x_ranges_value = x_ranges_value_left_to_right
elif verse == 'right_to_left':
  x_ranges_value = x_ranges_value_right_to_left

x_ranges_value,y_ranges_value

([100.0,
  97.61904761904762,
  95.23809523809524,
  92.85714285714286,
  90.47619047619048,
  88.09523809523809,
  85.71428571428571,
  83.33333333333333,
  80.95238095238095,
  78.57142857142857,
  76.19047619047619,
  73.80952380952381,
  71.42857142857143,
  69.04761904761905,
  66.66666666666667,
  64.28571428571429,
  61.904761904761905,
  59.523809523809526,
  57.14285714285714,
  54.76190476190476,
  52.38095238095238,
  50.0,
  47.61904761904762,
  45.23809523809524,
  42.857142857142854,
  40.476190476190474,
  38.095238095238095,
  35.714285714285715,
  33.333333333333336,
  30.952380952380953,
  22,
  20,
  19,
  17,
  15,
  13,
  11,
  9,
  4,
  3,
  2,
  1],
 [67.26852346349645,
  60.78704198201497,
  54.305560500533495,
  47.824079019052014,
  41.342597537570526,
  34.861116056089045,
  28.379634574607564,
  21.898153093126098,
  15.416671611644603,
  8.935190130163122,
  2.4537086486816406,
  -4.02777283279984,
  -10.509254314281321,
  -16.990735795762816,
  -23.4722172

In [239]:
def probabilityPlusMinus(x):
  
  if x[-1] == opponent.title():
    return list(x[:3])+[-float(x[3])]+list(x[4:])+[opponent_verse]
  else:
    return list(x)+[verse]
  
def opponentCount(x):
  
  try:
    team_1_prob = float(x[3]) 
  except:
    team_1_prob = 0.0

  try:
    team_2_prob = float(x[5]) 
  except:
    team_2_prob = 0.0

  if team_1_prob > team_2_prob:
    return [x[0],x[1],x[2],x[3],-x[6]]
  else:
    return [x[0],x[1],x[2],x[5],-x[4]]

pre_max_pitch_pivot_gr = df.where(df['team'] != '').groupBy('target_x','target_y','time').pivot('team').agg(func.max('probability'),func.count('player')).fillna(0.0)

max_pitch_pivot = spark.createDataFrame(pre_max_pitch_pivot_gr.rdd.map(lambda a : opponentCount(a)),['target_x','target_y','time','probability','opponents_count'])

max_pitch_pivot.show()

+--------+--------+------+------------------+---------------+
|target_x|target_y|  time|       probability|opponents_count|
+--------+--------+------+------------------+---------------+
|   -10.0|    11.0|4315.0| 67.47384920630097|              0|
|   -13.0|   -34.0|2763.0|             100.0|              0|
|   -13.0|     8.0|2763.0|61.408083447399186|              0|
|   -14.0|    12.0|4315.0|54.914148146508566|              0|
|   -15.0|    -3.0|3291.0| 57.64168914438462|             -1|
|   -15.0|     7.0|2763.0| 67.16457808313513|              0|
|   -16.0|   -31.0|2763.0|             100.0|              0|
|   -16.0|     3.0|3291.0| 28.83203022313935|             -2|
|   -17.0|     8.0|4315.0|             100.0|              0|
|    -2.0|    12.0|2763.0|             100.0|              0|
|   -21.0|   -19.0|4315.0|             100.0|              0|
|   -22.0|   -30.0|2763.0|             100.0|              0|
|   -23.0|   -23.0|2763.0|             100.0|              0|
|   -24.

In [240]:
max_player = df.select('target_x','target_y','time','player','team','probability','distance_from_target','x','y','slope_fuzzy_compare','ball_distance')

In [241]:
max_pitch_pivot_j = max_pitch_pivot.join(max_player, ['target_x','target_y','time','probability'])

max_pitch_pivot_jj = spark.createDataFrame(max_pitch_pivot_j.rdd.map(lambda f : probabilityPlusMinus(f)),max_pitch_pivot_j.columns+['verse'])
max_pitch_pivot_jj = max_pitch_pivot_jj.withColumn('probability', max_pitch_pivot_jj['probability'].cast(DoubleType())).fillna(0)
pre_max_pitch_pivot_jjj = max_pitch_pivot_jj.where(max_pitch_pivot_jj['team'] == team_a.capitalize())

In [242]:
max_pitch_pivot_jj.groupBy('player','team').avg('probability').show()

+------------+--------+------------------+
|      player|    team|  avg(probability)|
+------------+--------+------------------+
|    Florenzi|   Milan| 53.90035435104492|
|      Deiola|Cagliari|60.156671762349184|
|       Kjaer|   Milan| 69.93511694176463|
|    Calabria|   Milan| 87.00750199844386|
|       Marin|Cagliari| 70.76216223138441|
|   Strootman|Cagliari| 57.62370212904159|
|      Giroud|   Milan| 75.39965015086739|
|       Zappa|Cagliari| 55.02397863127716|
|        Diaz|   Milan| 67.87182504408166|
|       Rebic|   Milan| 70.23229383205116|
|    Bennacer|   Milan| 72.97034582147238|
|        Leao|   Milan| 71.15637130251879|
|   Pavoletti|Cagliari| 64.23935597390783|
|     Carboni|Cagliari| 81.79938713940078|
|      Krunic|   Milan| 71.21380154027291|
|      Tonali|   Milan| 67.83545491927318|
|       Godin|Cagliari| 89.57181554591442|
|      Tomori|   Milan|48.391163369783236|
|  Joao Pedro|Cagliari|48.342833147920864|
|Saelemaekers|   Milan|  76.0394848092751|
+----------

In [243]:
pd_probs = pre_max_pitch_pivot_jjj.toPandas()

In [244]:
def findBestMaxObs(x,panda,max_obs):
  
  probs = panda[panda['player'] == x[0]][panda['time'] == x[1]][['probability','distance_from_target']].values.tolist()
  
  try:
    first_twenty_probs = sorted(list(map(lambda a : a[0],probs)), reverse = True)[:max_obs]
    prob_twentieth = first_twenty_probs[-1]
    distance_twentieth = 100.0

    if float(prob_twentieth) == 100.0:
      first_twenty_distances = sorted(list(map(lambda g : g[1], list(filter(lambda a : a[0] == 100.0,probs)))))[:max_obs]
      distance_twentieth = first_twenty_distances[-1]

    return [prob_twentieth,distance_twentieth]
  except:
    return [0.0,0.0]

players_times_values = spark.createDataFrame(df.where(df['team'] == team_a.title()).select('player','time').distinct().rdd.map(lambda a : list(a)+findBestMaxObs(a,pd_probs,60)),['player','time','probability_twentieth','distance_twentieth'])

players_times_values.show()

+------------+------+---------------------+------------------+
|      player|  time|probability_twentieth|distance_twentieth|
+------------+------+---------------------+------------------+
|      Tomori|3291.0|    55.87081289098867|             100.0|
|        Diaz|4315.0|   21.296459907529368|             100.0|
|      Giroud|3291.0|                100.0|              null|
|    Bennacer|4315.0|                100.0|              null|
|    Calabria|4315.0|                100.0|              null|
|       Rebic|4315.0|                100.0|              null|
|   Hernandez|2763.0|   42.116505130145725|             100.0|
|     Maignan|4315.0|                100.0|              null|
|      Krunic|3291.0|                100.0|              null|
|   Hernandez|4315.0|    42.19503016000804|             100.0|
|      Tonali|3291.0|                100.0|              null|
|      Tomori|2763.0|   53.372267225712235|             100.0|
|      Krunic|2763.0|                  0.0|            

In [245]:
pd_to_filter = players_times_values.toPandas()

In [246]:
def obsToFilter(x, panda):
  
  prob_filter, distance_filter = panda[panda['player'] == x[5]][panda['time'] == x[2]][['probability_twentieth','distance_twentieth']].values.tolist()[0]
  
  flag = ''
  
  if (x[3] < prob_filter) or (x[7] > distance_filter):
    flag = 'out'
  
  return flag
  
max_pitch_pivot_jjj_pre_filt = spark.createDataFrame(pre_max_pitch_pivot_jjj.rdd.map(lambda a : list(a)+[obsToFilter(a, pd_to_filter)]), pre_max_pitch_pivot_jjj.columns+['obs_flag'])

max_pitch_pivot_jjj = max_pitch_pivot_jjj_pre_filt.where(max_pitch_pivot_jjj_pre_filt['obs_flag'] == '').where(max_pitch_pivot_jjj_pre_filt['probability'] > 0).drop('obs_flag')

In [247]:
#display(max_pitch_pivot_jjj.where(max_pitch_pivot_jjj['target_x'] == 42.0).where(max_pitch_pivot_jjj['target_y'] == 5.0).where(max_pitch_pivot_jjj['time'] == 1135.0))

In [248]:
from pyspark.sql.functions import lit

def ifHigherThanLimit(a,limit,maxMinFlag): 
  if maxMinFlag == 'max':
    return float(np.min([limit,a]))
  elif maxMinFlag == 'min':
    return float(np.max([limit,a]))

def addFakeProb(team):
  
  if team == team_a.title():
    return 150.0
  else:
    return -150.0

  
### PITCH ###
pitch_pivot = max_pitch_pivot_jj.select('target_x','target_y','time','probability','player','team')
pitch_pivot = pitch_pivot.withColumn('type',lit('pitch'))
pitch_pivot = pitch_pivot.withColumnRenamed('target_x','x').withColumnRenamed('target_y','y')

### PLAYERS ###
players_locs = spark.createDataFrame(data_players.select('player','team','time','x','y').withColumn('type',lit('player')).distinct().rdd.map(lambda a : list(a)+[addFakeProb(a[1])]), ['player','team','time','x','y','type','probability'])
players_locs = players_locs.where(players_locs['time'].isin(timings))

### BALL ###
ball = pre_pre_df.where(pre_pre_df['target_x'] == -999.0).orderBy('time')
ball_locs = ball.withColumn('probability',lit(200.0)).withColumn('type',lit('ball')).select('x','y','time','probability','player','team','type')
pd_ball = ball.toPandas()

In [249]:
extra_ball_locs = ball_locs.select('time','x','y').distinct().withColumnRenamed('x','x_ball').withColumnRenamed('y','y_ball')
extra_ball_locs = extra_ball_locs.withColumn('verse', lit(verse))

In [250]:
def addNegativeProb(x):
  
  team = x[team_index]
  probability = x[prob_index]
  
  if team == opponent.title():
    new_prob = -float(probability)
  else:
    new_prob = float(probability)
    
  y = list(x[:prob_index])+[new_prob]+list(x[1+prob_index:])
  
  return y
  
pitch_pivot_union = pitch_pivot.distinct()

team_index, prob_index = pitch_pivot_union.columns.index('team'),pitch_pivot_union.columns.index('probability')

pitch_pivot_union = spark.createDataFrame(pitch_pivot_union.rdd.map(lambda y : addNegativeProb(y)), pitch_pivot_union.columns)

for i in players_locs.columns:
  if 'class' in str(i):
    pitch_pivot_union = pitch_pivot_union.withColumn(str(i), lit(''))
    
out = pitch_pivot_union.union(players_locs.select(pitch_pivot_union.columns)).union(ball_locs.select(pitch_pivot_union.columns))

In [251]:
out_with_extra_ball = out.join(extra_ball_locs, ['time'])

In [252]:
new_path = 'pitch_control_2_'+name_path
new_path

'pitch_control_2_milan_cagliari_first_half_2_all'

In [253]:
temp_path = "/content/drive/MyDrive/Colab Notebooks/FootballDataAnalysis/data/Milan-Cagliari/parquet/__temp"
target_path = "/content/drive/MyDrive/Colab Notebooks/FootballDataAnalysis/data/Milan-Cagliari/parquet/"+new_path+".csv"

out.coalesce(1).write.mode("overwrite").option('delimiter',',').option('header','true').csv(temp_path)

Path = spark.sparkContext._gateway.jvm.org.apache.hadoop.fs.Path

# get the part file generated by spark write
fs = Path(temp_path).getFileSystem(spark.sparkContext._jsc.hadoopConfiguration())
csv_part_file = fs.globStatus(Path(temp_path + "/part*"))[0].getPath()

# move and rename the file
fs.rename(csv_part_file, Path(target_path))
fs.delete(Path(temp_path), True)

True

In [254]:
out_with_extra_ball.coalesce(1).write.option('delimiter',',').option('header','true').csv('/content/drive/MyDrive/Colab Notebooks/FootballDataAnalysis/data/Milan-Cagliari/parquet/'+new_path+'.csv', mode='overwrite')

In [255]:
#pd_pitch = max_pitch_pivot_jj.toPandas()

In [256]:
'''
from bisect import bisect
from pyspark.sql.window import Window

def areaDanger(d,x_ranges,y_ranges):
  
  if d[-1] == 'right_to_left':
    x_values = x_ranges_value_right_to_left
  else:
    x_values = x_ranges_value_left_to_right
  
  try:
    x_value = x_values[bisect(x_ranges,d[0])]
  except:
    try:
      x_value = x_values[bisect(x_ranges,d[0])-1]
    except:
      x_value = x_values[bisect(x_ranges,d[0])+1]
      
  try:
    y_value = y_ranges_value[bisect(y_ranges,d[1])]
  except:
    try:
      y_value = y_ranges_value[bisect(y_ranges,d[1])-1]
    except:
      y_value = y_ranges_value[bisect(y_ranges,d[1])+1]
  
  if x_value < 50:
    y_value = 10
  
  return [bisect(x_ranges,d[0]),bisect(y_ranges,d[1]),float(np.round(x_value,0)),float(np.round(y_value,0)),float(np.round(x_value+y_value,0))]
 
def findOpponentsOvercomeByThePass(x,panda,panda_possess,pd_pitch):
  
  player_x = x[x_index]
  player_y = x[y_index]
  verse = x[verse_index]
  time_ = x[time_index]
  target_x = x[target_x_index]
  target_y = x[target_y_index]
  
  player_possess,possess_x,possess_y = panda_possess[panda_possess['time'] == time_][['player_in_possess','x','y']].values.tolist()[0]
  
  sit_x = 0.25*player_x+0.75*target_x
  sit_y = 0.25*player_y+0.75*target_y

  x_advancement_flag = float(np.sign(opponent_goal_line[0] - target_x))

  values_x = [sit_x-(x_advancement_flag*10),sit_x+(x_advancement_flag*10)]

  back_min_x,ahead_max_x = float(np.min(values_x)),float(np.max(values_x))

  up_y = sit_y+10
  down_y = sit_y-10

  quarters_control_percentages = []

  close_opponents_panda = panda[panda['team'] == opponent.title()][panda['time'] == time_][panda['x'] > player_x-4.0][panda['x'] < player_x+4.0][panda['y'] > player_y-4.0][panda['x'] < player_y+4.0]
  close_opponents = len(close_opponents_panda)
  opponents_x_y = panda[panda['team'] == opponent.title()][panda['time'] == time_][panda['player'] != opponent_goalies][['x','y']].values.tolist()
  
  close_opponents_distances = []
  for j in opponents_x_y:
    close_opponents_distances.append(float(np.sqrt((player_x-j[0])**2+(player_y-j[1])**2)))
     
  closest_opponent_distance = sorted(close_opponents_distances)[0]
  player_distance_from_goal_line = float(np.sqrt((player_x - opponent_goal_line[0])**2 + (player_y - opponent_goal_line[1])**2))
  
  opponents_distances_to_goal_line = []
  for j in opponents_x_y:
    opponents_distances_to_goal_line.append(float(np.sqrt((opponent_goal_line[0]-j[0])**2+(opponent_goal_line[1]-j[1])**2)))
    
  opponents_from_goal_line = len(list(filter(lambda a : a < player_distance_from_goal_line-1.0, opponents_distances_to_goal_line)))
    
  return [closest_opponent_distance,opponents_from_goal_line,close_opponents,player_distance_from_goal_line]
  
verse_index = max_pitch_pivot_jjj.columns.index('verse')
time_index = max_pitch_pivot_jjj.columns.index('time')
target_x_index = max_pitch_pivot_jjj.columns.index('target_x')
target_y_index = max_pitch_pivot_jjj.columns.index('target_y')
y_index = max_pitch_pivot_jjj.columns.index('y')
x_index = max_pitch_pivot_jjj.columns.index('x')

ppp_area_targets = sqlContext.createDataFrame(max_pitch_pivot_jjj.where((max_pitch_pivot_jjj['player'].isin([team_a_goalies])) == False).rdd.map(lambda g : list(g)+areaDanger(g,x_ranges,y_ranges)+findOpponentsOvercomeByThePass(g,panda_locs,panda_possess,pd_pitch)), max_pitch_pivot_jjj.columns+['x_area','y_area','x_value','y_value','area_value','closest_opponent_distance','opponents_from_goal_line','close_opponents','player_distance_from_goal_line'])

area_targets_advance_pre_final = ppp_area_targets.select('target_x','target_y','player','time','area_value','ball_distance','closest_opponent_distance','opponents_from_goal_line','distance_from_target','x','y','probability','close_opponents','player_distance_from_goal_line','slope_fuzzy_compare').distinct()
'''

"\nfrom bisect import bisect\nfrom pyspark.sql.window import Window\n\ndef areaDanger(d,x_ranges,y_ranges):\n  \n  if d[-1] == 'right_to_left':\n    x_values = x_ranges_value_right_to_left\n  else:\n    x_values = x_ranges_value_left_to_right\n  \n  try:\n    x_value = x_values[bisect(x_ranges,d[0])]\n  except:\n    try:\n      x_value = x_values[bisect(x_ranges,d[0])-1]\n    except:\n      x_value = x_values[bisect(x_ranges,d[0])+1]\n      \n  try:\n    y_value = y_ranges_value[bisect(y_ranges,d[1])]\n  except:\n    try:\n      y_value = y_ranges_value[bisect(y_ranges,d[1])-1]\n    except:\n      y_value = y_ranges_value[bisect(y_ranges,d[1])+1]\n  \n  if x_value < 50:\n    y_value = 10\n  \n  return [bisect(x_ranges,d[0]),bisect(y_ranges,d[1]),float(np.round(x_value,0)),float(np.round(y_value,0)),float(np.round(x_value+y_value,0))]\n \ndef findOpponentsOvercomeByThePass(x,panda,panda_possess,pd_pitch):\n  \n  player_x = x[x_index]\n  player_y = x[y_index]\n  verse = x[verse_index]\n 

In [257]:
#display(area_targets_advance_pre_final.where(area_targets_advance_pre_final['time'] == 3981))

In [258]:
'''
def singlePointOpponents(x,y,panda_locs,opponent,time_,meters_to_intercept,possess_x,possess_y):
  
  distance = distance = float(np.sqrt(((x-possess_x)**2)+((y-possess_y)**2)))
  
  opponent_meters_run = 3*(float(distance)/18.0)
  opps =  panda_locs[panda_locs['team'] == opponent.title()][panda_locs['time'] == time_][panda_locs['x'] > x-meters_to_intercept-opponent_meters_run][panda_locs['x'] < x+meters_to_intercept+opponent_meters_run][panda_locs['y'] > y-meters_to_intercept-opponent_meters_run][panda_locs['y'] < y+meters_to_intercept+opponent_meters_run]['player_id'].values.tolist()
  
  try:
    return opps[0]
  except:
    return []
  
def interceptionOpponents(player,time_,target_x,target_y,x,y,panda,panda_possess):
  
  player_possess,possess_x,possess_y = panda_possess[panda_possess['time'] == time_][['player_in_possess','x','y']].values.tolist()[0]

  interception_opponents = []
  interception_opponents_count = len(interception_opponents)
  
  if str(player) == str(player_possess):
    pass
    
  else:
    distance = float(np.sqrt(((target_x-possess_x)**2)+((target_y-possess_y)**2)))

    m = (possess_y-target_y)/(possess_x-target_x)

    b = (possess_x*target_y - target_x*possess_y)/(possess_x-target_x)

    if possess_x > target_x:
      xs = sorted(list(np.arange(target_x,possess_x,float(possess_x-target_x)/distance)), reverse = True)
    else:
      xs = list(np.arange(possess_x,target_x,float(target_x-possess_x)/distance))

    pre_trajectory = list(map(lambda xx: [float(xx),float((m*xx)+b)], xs))

    if distance > 25:
      trajectory = [pre_trajectory[2],pre_trajectory[3],pre_trajectory[4],pre_trajectory[5],pre_trajectory[6],pre_trajectory[-6],pre_trajectory[-5],pre_trajectory[-4],pre_trajectory[-3],pre_trajectory[-2],pre_trajectory[-1]]
    else:
      trajectory = pre_trajectory[2:]

    interception_opponents = list(filter(lambda a : a!= [], list(map(lambda d : singlePointOpponents(d[0],d[1],panda,opponent,time_,1.0,possess_x,possess_y), trajectory))))
    interception_opponents_count = len(list(dict.fromkeys(interception_opponents)))
    
  return [interception_opponents_count]


player_index = area_targets_advance_pre_final.columns.index('player')
time_index = area_targets_advance_pre_final.columns.index('time')
target_x_index = area_targets_advance_pre_final.columns.index('target_x')
target_y_index = area_targets_advance_pre_final.columns.index('target_y')
x_index = area_targets_advance_pre_final.columns.index('x')
y_index = area_targets_advance_pre_final.columns.index('y')

area_targets_advance_pre_final_intercept = sqlContext.createDataFrame(area_targets_advance_pre_final.rdd.map(lambda g : list(g)+interceptionOpponents(g[player_index],g[time_index],g[target_x_index],g[target_y_index],g[x_index],g[y_index],panda_locs,panda_possess)), area_targets_advance_pre_final.columns+['interception_opponents'])
'''


"\ndef singlePointOpponents(x,y,panda_locs,opponent,time_,meters_to_intercept,possess_x,possess_y):\n  \n  distance = distance = float(np.sqrt(((x-possess_x)**2)+((y-possess_y)**2)))\n  \n  opponent_meters_run = 3*(float(distance)/18.0)\n  opps =  panda_locs[panda_locs['team'] == opponent.title()][panda_locs['time'] == time_][panda_locs['x'] > x-meters_to_intercept-opponent_meters_run][panda_locs['x'] < x+meters_to_intercept+opponent_meters_run][panda_locs['y'] > y-meters_to_intercept-opponent_meters_run][panda_locs['y'] < y+meters_to_intercept+opponent_meters_run]['player_id'].values.tolist()\n  \n  try:\n    return opps[0]\n  except:\n    return []\n  \ndef interceptionOpponents(player,time_,target_x,target_y,x,y,panda,panda_possess):\n  \n  player_possess,possess_x,possess_y = panda_possess[panda_possess['time'] == time_][['player_in_possess','x','y']].values.tolist()[0]\n\n  interception_opponents = []\n  interception_opponents_count = len(interception_opponents)\n  \n  if str(play

In [259]:
'''
spark.conf.set("spark.sql.legacy.allowCreatingManagedTableUsingNonemptyLocation","true")
out.write.saveAsTable("pitch_control_2_"+str(team_a_lower)+"_"+str(opponent_lower)+"_"+str(half).replace(' ','_')+str(half_split)+"_"+str(player_in_possess), mode = 'overwrite') 
'''

'\nspark.conf.set("spark.sql.legacy.allowCreatingManagedTableUsingNonemptyLocation","true")\nout.write.saveAsTable("pitch_control_2_"+str(team_a_lower)+"_"+str(opponent_lower)+"_"+str(half).replace(\' \',\'_\')+str(half_split)+"_"+str(player_in_possess), mode = \'overwrite\') \n'

In [260]:
'''
spark.conf.set("spark.sql.legacy.allowCreatingManagedTableUsingNonemptyLocation","true")
area_targets_advance_pre_final_intercept.write.saveAsTable("pitch_control_2_kpis_marcature_"+str(team_a_lower)+"_"+str(opponent_lower)+"_"+str(half).replace(' ','_')+str(half_split)+"_"+str(player_in_possess), mode = 'overwrite')
''' 

'\nspark.conf.set("spark.sql.legacy.allowCreatingManagedTableUsingNonemptyLocation","true")\narea_targets_advance_pre_final_intercept.write.saveAsTable("pitch_control_2_kpis_marcature_"+str(team_a_lower)+"_"+str(opponent_lower)+"_"+str(half).replace(\' \',\'_\')+str(half_split)+"_"+str(player_in_possess), mode = \'overwrite\')\n'

In [261]:
#str(half).replace(' ','_')+str(half_split)

In [262]:
'''
# area_value
area_targets_advance_pre_final_area_value = area_targets_advance_pre_final.groupBy('player_id','time').agg(func.avg('area_value'),func.max('area_value')).withColumnRenamed('avg(area_value)','avg').withColumnRenamed('max(area_value)','max').withColumn('kpi', lit('area_value'))
area_targets_advance_pre_final_area_value = area_targets_advance_pre_final_area_value.withColumn('area_value_index', func.log((1+(0.45*area_targets_advance_pre_final_area_value['avg']+0.55*area_targets_advance_pre_final_area_value['max']))/area_targets_advance_pre_final_area_value['avg'])).select('player_id','time','area_value_index')
# opponents_cut
area_targets_advance_pre_final_opponents_cut = area_targets_advance_pre_final.groupBy('player_id','time').agg(func.avg('opponents_cut'),func.max('opponents_cut')).withColumnRenamed('avg(opponents_cut)','avg').withColumnRenamed('max(opponents_cut)','max').withColumn('kpi', lit('opponents_cut'))
area_targets_advance_pre_final_opponents_cut = area_targets_advance_pre_final_opponents_cut.withColumn('opponents_cut_index', func.log((1+(0.45*area_targets_advance_pre_final_opponents_cut['avg']+0.55*area_targets_advance_pre_final_opponents_cut['max']))/area_targets_advance_pre_final_opponents_cut['avg'])).select('player_id','time','opponents_cut_index')
# percentage_zone_in_control
area_targets_advance_pre_final_percentage_zone_in_control = area_targets_advance_pre_final.groupBy('player_id','time').agg(func.avg('percentage_zone_in_control'),func.max('percentage_zone_in_control')).withColumnRenamed('avg(percentage_zone_in_control)','avg').withColumnRenamed('max(percentage_zone_in_control)','max').withColumn('kpi', lit('percentage_zone_in_control'))
area_targets_advance_pre_final_percentage_zone_in_control = area_targets_advance_pre_final_percentage_zone_in_control.withColumn('percentage_zone_in_control_index', func.log((1+(0.45*area_targets_advance_pre_final_percentage_zone_in_control['avg']+0.55*area_targets_advance_pre_final_percentage_zone_in_control['max']))/area_targets_advance_pre_final_percentage_zone_in_control['avg'])).select('player_id','time','percentage_zone_in_control_index')
# team_advancement
area_targets_advance_pre_final_team_advancement = area_targets_advance_pre_final.groupBy('player_id','time').agg(func.avg('team_advancement'),func.max('team_advancement')).withColumnRenamed('avg(team_advancement)','avg').withColumnRenamed('max(team_advancement)','max').withColumn('kpi', lit('team_advancement'))
area_targets_advance_pre_final_team_advancement = area_targets_advance_pre_final_team_advancement.withColumn('team_advancement_index', func.log((1+(0.45*area_targets_advance_pre_final_team_advancement['avg']+0.55*area_targets_advance_pre_final_team_advancement['max']))/area_targets_advance_pre_final_team_advancement['avg'])).select('player_id','time','team_advancement_index')
# distance_from_target
area_targets_advance_pre_final_distance_from_target = area_targets_advance_pre_final.groupBy('player_id','time').agg(func.avg('distance_from_target'),func.max('distance_from_target')).withColumnRenamed('avg(distance_from_target)','avg').withColumnRenamed('max(distance_from_target)','max').withColumn('kpi', lit('distance_from_target'))
area_targets_advance_pre_final_distance_from_target = area_targets_advance_pre_final_distance_from_target.withColumn('distance_from_target_index', func.log((1+(0.45*area_targets_advance_pre_final_distance_from_target['avg']+0.55*area_targets_advance_pre_final_distance_from_target['max']))/area_targets_advance_pre_final_distance_from_target['avg'])).select('player_id','time','distance_from_target_index')
# probability
area_targets_advance_pre_final_probability = area_targets_advance_pre_final.groupBy('player_id','time').agg(func.avg('probability'),func.max('probability')).withColumnRenamed('avg(probability)','avg').withColumnRenamed('max(probability)','max').withColumn('kpi', lit('probability'))
area_targets_advance_pre_final_probability = area_targets_advance_pre_final_probability.withColumn('probability_index', func.log((1+(0.45*area_targets_advance_pre_final_probability['avg']+0.55*area_targets_advance_pre_final_probability['max']))/area_targets_advance_pre_final_probability['avg'])).select('player_id','time','probability_index')
# ball_distance
area_targets_advance_pre_final_ball_distance = area_targets_advance_pre_final.groupBy('player_id','time').agg(func.avg('ball_distance'),func.max('ball_distance')).withColumnRenamed('avg(ball_distance)','avg').withColumnRenamed('max(ball_distance)','max').withColumn('kpi', lit('ball_distance'))
area_targets_advance_pre_final_ball_distance = area_targets_advance_pre_final_ball_distance.withColumn('ball_distance_index', func.log((1+(0.45*area_targets_advance_pre_final_ball_distance['avg']+0.55*area_targets_advance_pre_final_ball_distance['max']))/area_targets_advance_pre_final_ball_distance['avg'])).select('player_id','time','ball_distance_index')

kpis = area_targets_advance_pre_final_area_value.join(area_targets_advance_pre_final_opponents_cut, ['player_id','time']).join(area_targets_advance_pre_final_percentage_zone_in_control, ['player_id','time']).join(area_targets_advance_pre_final_team_advancement, ['player_id','time']).join(area_targets_advance_pre_final_distance_from_target, ['player_id','time']).join(area_targets_advance_pre_final_probability, ['player_id','time']).join(area_targets_advance_pre_final_ball_distance, ['player_id','time'])
'''

"\n# area_value\narea_targets_advance_pre_final_area_value = area_targets_advance_pre_final.groupBy('player_id','time').agg(func.avg('area_value'),func.max('area_value')).withColumnRenamed('avg(area_value)','avg').withColumnRenamed('max(area_value)','max').withColumn('kpi', lit('area_value'))\narea_targets_advance_pre_final_area_value = area_targets_advance_pre_final_area_value.withColumn('area_value_index', func.log((1+(0.45*area_targets_advance_pre_final_area_value['avg']+0.55*area_targets_advance_pre_final_area_value['max']))/area_targets_advance_pre_final_area_value['avg'])).select('player_id','time','area_value_index')\n# opponents_cut\narea_targets_advance_pre_final_opponents_cut = area_targets_advance_pre_final.groupBy('player_id','time').agg(func.avg('opponents_cut'),func.max('opponents_cut')).withColumnRenamed('avg(opponents_cut)','avg').withColumnRenamed('max(opponents_cut)','max').withColumn('kpi', lit('opponents_cut'))\narea_targets_advance_pre_final_opponents_cut = area_ta